In [2]:
import os
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

设定用于训练和评估的样本总数
num_examples_pre_epoch_for_train = 50000
num_examples_pre_epoch_for_eval = 10000
定义读取cifar10的函数 ----->读取目标文件的内容
定义数据预处理的函数 ----->图像增强

In [4]:
def read_cifar10(file_queue):
    #用来读取cifar10图片信息并进行切割区分
    
    class CIFAR10Record(object):
        pass
    #定义一个空类，用于返回读取的Cifar10数据
    result = CIFAR10Record()
    
    label_bytes = 1
    result.height = 32
    result.width = 32
    result.channel = 3
    #输入的图像数据 label bytes由于分类为10所以占位是1（0-9）
    
    images_bytes = result.height * result.width * result.channel
    record_bytes = label_bytes+images_bytes
    #计算图像占位和总体占位
    
    reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
    #读取固定长度

    result.key, value = reader.read(filename_queue) #从inputs函数中获取的filename_queue
    record_bytes=tf.decode_raw(value,tf.uint8) 
    
    result.label = tf.strided_slice(record_bytes,[0], [label_bytes])
    result.label = tf.cast(result.label, tf.int32)

    result.image = tf.strided_slice(record_bytes,[label_bytes],[label_bytes + image_bytes] )
    result.image = tf.reshape(result.channel, result.height, result.width)
    result.image = tf.transpose(result.image, [1,2,0])
    
    return result


In [1]:
def inputs(data_dir, batch_size, distorted): #需要输入，数据集地址，批大小，distorted= True or False
    

    filename = [os.path.join(data_dir, 'data_batch_%d.bin'%i)for i in range(1,6)]
    #['Cifar_data/cifar-10-batches-bin\\data_batch_1.bin', 
    #'Cifar_data/cifar-10-batches-bin\\data_batch_2.bin',
    #'Cifar_data/cifar-10-batches-bin\\data_batch_3.bin', 
    #'Cifar_data/cifar-10-batches-bin\\data_batch_4.bin', 
    #'Cifar_data/cifar-10-batches-bin\\data_batch_5.bin']
    
    filename_queue = tf.train.string_input_producer(filenames, shuffle=True)
    #创建输入队列
    
    read_input = read_cifar10(filename_queue)
    #根据自定义read_cifar10函数来读取数据集信息
    
    reshaped_image = tf.cast(read_input.unit8image, tf.float32)
    #将读取的信息转换成float32格式
    
    num_examples_per_epoch = num_examples_pre_epoch_for_train
    #指定用于训练的图像数量
    
    if distorted != None: #如果distorted不为空，则进行数据增强处理
        '''
        数据剪切，tf.random.crop
        左右翻转 tf.image.random_flip_left_right()
        随机对比度调整 tf.image.random_contrast()
        标准化图片操作tf.image.per_image_standardization()
        '''
        cropped_image = tf.random.crop(reshaped_image, [24,24,3])
        flipped_image = tf.image.random_flip_left_right(cropped_image)
        adjusted_image = tf.image.random_contrast(flipped_image, max_delta = 0.8)
        lightten_image = tf.image.random_brightness(adjusted_image, lower=0.2, upper=0.8)
        normalized_image = tf.image.per_image_standardization(lightten_image)
        
        normalized_image.set_shae([24,24,3])
        read_input.label.set_shape([1])
        
        min_queue_examples = int(num_examples_pre_epoch_for_eval*0.4)
        print('')
        
        images_train, labels_train = tf.train.shuffle_batch([normalized_image, read_input.label], 
                                                                batch_size=batch_size, num_threads=16,
                                                                capacity = min_queue_examples + 3*batch_size,
                                                                min_after_dequeue = min_queue_examples)
        
        return image_train, tf.reshape(labels_train, [batch_size])
    
    else:
        #测试图片不需要进行数据增强，但是需要resize和normalize
        resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,24,24)
        normalized_image = tf.image.per_image_standarlization(resized_image)
        
        normalized_image.set_shape([24,24,3])
        read_input.label.set_shape([1])
        
        min_queue_examples = int(num_examples_pre_epoch_for_eval*0.4)
        print('')
        
        images_test,labels_test = tf.train.batch([normalized_image, read_input.label], 
                                                                batch_size=batch_size, num_threads=16,
                                                                capacity = min_queue_examples + 3*batch_size,
                                                                min_after_dequeue = min_queue_examples)
        
        return images_test, tf.reshape(labels_test, [batch_size])        

In [3]:
data_dir="Cifar_data/cifar-10-batches-bin"
filenames=[os.path.join(data_dir,"data_batch_%d.bin"%i)for i in range(1,6)]
print(filenames)

['Cifar_data/cifar-10-batches-bin\\data_batch_1.bin', 'Cifar_data/cifar-10-batches-bin\\data_batch_2.bin', 'Cifar_data/cifar-10-batches-bin\\data_batch_3.bin', 'Cifar_data/cifar-10-batches-bin\\data_batch_4.bin', 'Cifar_data/cifar-10-batches-bin\\data_batch_5.bin']
